### 构建路由链

In [1]:

from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate

import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("api_key")
base_url = os.getenv("base_url")

online_llm = ChatOpenAI(
    api_key=api_key,
    model="qwen-plus",
    base_url=base_url,
    temperature=0
)







In [6]:
# 路由链相关环境
from langchain.chains import LLMChain, ConversationChain
from langchain.chains.router import MultiRouteChain, LLMRouterChain, MultiPromptChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser

In [3]:
# 底层 目标链的prompt
math_prompt = """
你是一位数学家，请根据以下问题，生成对应的数学公式.内容：{input}
"""

weather_prompt = """你是一位天气预报员，请根据以下问题，生成对应的天气信息.内容：{input}"""
news_prompt = """你是一位新闻工作者，请根据以下问题，生成对应的新闻内容.内容：{input}"""

In [4]:
#提示词模版 相关信息
prompt_info = [
    {
        "name": "math",
        "description": "数学问题",
        "prompt": math_prompt
    },
    {
        "name": "weather",
        "description": "天气问题",
        "prompt": weather_prompt
    },
    {
        "name": "news",
        "description": "新闻问题",
        "prompt": news_prompt
    }
]

In [5]:
#定义 目标链
chains = {}
for info in prompt_info:
    chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=info["prompt"]))
    chains[info["name"]] = chain

print(chains)

{'math': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n你是一位数学家，请根据以下问题，生成对应的数学公式.内容：{input}\n'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10cf81550>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10cac21d0>, root_client=<openai.OpenAI object at 0x10d3e1010>, root_async_client=<openai.AsyncOpenAI object at 0x10d604390>, model_name='qwen-plus', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1'), output_parser=StrOutputParser(), llm_kwargs={}), 'weather': LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='你是一位天气预报员，请根据以下问题，生成对应的天气信息.内容：{input}'), llm=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10cf81550>, async_clien

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_18361/2015125349.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=online_llm, prompt=PromptTemplate.from_template(template=info["prompt"]))


In [7]:
#定义默认链
default_chain = ConversationChain(llm=online_llm,output_key="text")

/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_18361/3167257219.py:2: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  default_chain = ConversationChain(llm=online_llm,output_key="text")
/Users/zhangtian/miniforge3/envs/langchain/lib/python3.11/site-packages/pydantic/main.py:214: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


In [8]:
#路由配置
prompt_info = [
    "math:你是一位数学家，请根据以下问题，生成对应的数学公式",
    "weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息",
    "news:你是一位新闻工作者，请根据以下问题，生成对应的新闻内容",
]

In [9]:
#构建路由模版


router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations = "\n".join(prompt_info))
print("路由模版是：",router_template)

router_prompt = PromptTemplate(
    template=router_template,
    # template="\n".join(prompt_info),
    input_variables=["input"],
    output_parser=RouterOutputParser()
    )
print("路由模版router_prompt 是：",router_prompt)

路由模版是： Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
math:你是一位数学家，请根据以下问题，生成对应的数学公式
weather:你是一位天气预报员，请根据以下问题，生成对应的天气信息
news:你是一位新闻工作者，请根据以下问题，生成对应

In [11]:
#构建路由链
router_chain = LLMRouterChain.from_llm(llm=online_llm, prompt=router_prompt, verbose=True)

In [12]:
#构建最终路由链，将路由链和默认链组合
expert_chain = MultiPromptChain(
    router_chain=router_chain, #路由链
    destination_chains=chains, #路由链对应的链
    default_chain=default_chain, #默认链
    verbose=True
)


/var/folders/xh/fl5j9g1n5t79htj8yk1cswzh0000gn/T/ipykernel_18361/2494765385.py:2: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  expert_chain = MultiPromptChain(


In [13]:
#调用最终链

question = [
    "今天北京天气怎么样？",
    "请给我一个数学题，简单一点的",
    "最近有什么新闻吗？",
    "你叫什么名字？",
    "请你写一首诗，歌颂祖国"
]

for q in question:
    print("问题：",q)
    print("回答：",expert_chain.invoke({"input":q}))

问题： 今天北京天气怎么样？


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
weather: {'input': '今天北京天气怎么样？'}
> Finished chain.
回答： {'input': '今天北京天气怎么样？', 'text': '大家好，这里是天气预报时间！接下来为您带来今天北京的天气情况：\n\n今天北京以晴间多云天气为主，白天阳光明媚，紫外线较强，建议市民外出时做好防晒措施。白天气温较高，预计最高气温将达到28℃左右，早晚较为舒适，最低气温约为16℃，昼夜温差较大，请适时增减衣物，以免着凉。\n\n此外，今天北京的空气质量良好，AQI指数在70左右，适合户外活动。不过，风力稍大，偏北风3-4级，阵风可达5级，请注意防风。总体来说，今天的天气适宜出行和各种户外活动，祝您拥有愉快的一天！\n\n以上就是今天北京的天气预报，感谢您的收听！'}
问题： 请给我一个数学题，简单一点的


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
math: {'input': '请生成一个简单的数学题'}
> Finished chain.
回答： {'input': '请生成一个简单的数学题', 'text': '当然！以下是一个简单的数学问题及其对应的数学公式：\n\n**问题：**  \n小明有 5 个苹果，他从市场上又买了 3 个苹果。请问小明一共有多少个苹果？\n\n**数学公式：**  \n$$\n\\text{总数} = 5 + 3\n$$\n\n如果需要更复杂的题目，请告诉我！'}
问题： 最近有什么新闻吗？


> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
news: {'input': '最近有什么新闻吗？'}
> Finished chain.
回答： {'input': '最